# Checking, Extracting and Ploting Respiration activity features

Single belt chest stretch measurements of respiration start with a wave that needs to be translated into features we can interprete. 

## Data Quality check
Check the raw respiration wave to evaluate whether it is catching respiratory waves clearly enough to be processed

## Extracting features - Inspiration onsets
Across all forms of respiration, inspiration is an active phase, the results of coordinated muscle contractions. Identifying the onsets of inspirations from the waves sometimes works well and sometimes is difficult to untangle from motion artifacts

## Extracting features - Respiration rate and Sequential differences
Changes in respiration rate can be a consequence of activity, attention, and affect. The feature of respiration rate is often a summary value for an interval but can also be tracked breath-by-breath.


In [ ]:
!git clone https://github.com/finn42/Beat_Breath_Workshop.git

Drag the qex.py module and the rp2.py into the main folder


In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import qex
import rp2

In [ ]:
demofile_loc = './Beat_Breath_Workshop/demodata/' #location for colab
df_datafiles = qex.qiosk_recordings(demofile_loc+'EQ/')
df_datafiles

# Extracting respiratory features, by breath
Note: this cell changes the index of the data opened to seconds since the first sample. The functions employed here depends on a dataframe index of Time (s), not datetime formate.

In [ ]:
recID = 5
V = pd.read_csv(df_datafiles.loc[recID,'RESPloc'])
V['DateTime'] = pd.to_datetime(V['DateTime'])
V['Time_s'] = (V['DateTime']-V.loc[0,'DateTime']).dt.total_seconds()

Resp = V.set_index('Time_s').copy()

# show the second-wise index, a rapid sample rate, 256 Hz
Resp.iloc[:10,:]

## Data Quality check

Reading the quality of measurements from a few samples is not so easy. The respiration wave is a unitless relative value, an estimate of chest circumference that is uncalibrated and adaptive to extremes in variation. The same values do not always correspond to the same circumference of chest or quantity of change between samples.

What we need to see is the contour of waves over time, showing the alternation of inspriration and expiration within a reasonable amount of time. 

In [ ]:
fig,(axes) = plt.subplots(1,2,figsize=(12,5),sharey=True)
Resp.loc[:60,'Breathing'].plot(ax = axes[0])
axes[0].grid()
axes[0].set(xlim=[0,60],ylabel = 'Respiration Wave', title = '60 seconds of respiration wave recording')
Resp['Breathing'].plot(ax = axes[1])
axes[1].grid()
axes[1].set(title = 'Respiration wave over full recording')

plt.show()


This looks like a respiratory wave, with fast increases (inspirations) and slower decreases and time in between. This measurement is from an interval of activity that produces irregular breathing cycles. 

## Extracting features - Inspiration onsets
The basis of tracking respiratory action is identify respiratory actions in the wave measurements. Here we use a heuristic algorithm to identify when inspirations begin (insp onset) and when expirations begin. With these moments identified, a host of other qualities about each breath can be evaluated.

In [ ]:
Inspirations = rp2.EQ_Inspiration_Extract(Resp['Breathing'])
Inspirations['In']

The inspiration onsets are reported in seconds 

Note: the little "Clipped!" notification that popped up is a warning that at least one respiratory cycle is clipped in this recording, meaning the shape of the top of a big breath was cutoff, obfuscating the onset of expiration a little. This happens with these Equivital vest measurements when people suddenly change posture and switch from more shallow quiet breathing to more extreme respiratory actions. Usually this clipping only happens a few times per recording.


In [ ]:
fig,(axes) = plt.subplots(1,2,figsize=(12,5),sharey=True)
Resp.loc[:60,'Breathing'].plot(ax = axes[0])
axes[0].grid()
axes[0].set(xlim=[0,60],ylabel = 'Respiration Wave', title = '60 seconds of respiration wave recording')

Resp.loc[:60,'Breathing'].plot(ax = axes[1])
axes[1].set(xticks = Inspirations['In'].values)
axes[1].set(xlim=[0,60],ylabel = 'Respiration Wave', title = '60 seconds of respiration wave with Insp Onsets')
axes[1].grid()
plt.show()

From these inspiration onsets we can evaluate the full respiratory cycles AND use these calculated breaths to normalise the recording a little to make measurements comparable between people. Here Breaths is a larger set of descriptive features and Insp_seq adds a number of sequential features, how one breath differs from the previous.

In [ ]:
Breaths = rp2.breath_cycles(rp2.EQ_Inspiration_Extract(Resp['Breathing']),Resp['Breathing'])
Insp_seq = rp2.quickcarresp(Breaths)
Insp_seq

We can chose which of these features to explore graphically or statistically, to evaluate respiratory behaviour during specific activities and conditions. 

Here are examples of plots from a few features.

In [ ]:
fig,(axes) = plt.subplots(3,1,figsize=(12,5),sharex=True)
axes[0].scatter(Insp_seq['In'],Insp_seq['In_C'],label = 'Inspiration onset')
axes[0].scatter(Insp_seq['Ex'],Insp_seq['Ex_C'],c='r',label = 'Expiration onset')
Resp['Breathing'].plot(ax = axes[0],label='Respiration wave')
axes[0].set(ylabel = 'Resp wave')
axes[0].grid()
axes[1].plot(Insp_seq['Ex'],Insp_seq[['Period_T','Exp_T']],label = ['Period_T','Exp_T'])
axes[1].plot(Insp_seq['In'],Insp_seq['Insp_T'],label = 'Insp_T')
axes[1].set(xlim=[0,60],ylabel='Duration (s)',xlabel='Time (s)')
axes[1].grid()
axes[1].legend()

axes[2].plot(Insp_seq['Ex'],Insp_seq[['dPT','relD']],label = ['Seq. Diff Period','Seq. Diff. Depth'])
axes[2].set(xlim=[0,60],ylabel='Ratios',xlabel='Time (s)')
axes[2].grid()
axes[2].legend()
plt.show()


Breathing in this interval is quite variable, some times short breaths, some times long, sometimes shallow, sometimes deep (and clipped!) while the breathing cycles towards the end are getting more consistently short. 

Lets look at another minute of this same recording.

In [ ]:
fig,(axes) = plt.subplots(3,1,figsize=(12,5),sharex=True)
axes[0].scatter(Insp_seq['In'],Insp_seq['In_C'],label = 'Inspiration onset')
axes[0].scatter(Insp_seq['Ex'],Insp_seq['Ex_C'],c='r',label = 'Expiration onset')
Resp['Breathing'].plot(ax = axes[0],label='Respiration wave')
axes[0].set(ylabel = 'Resp wave')

axes[0].grid()
axes[1].plot(Insp_seq['Ex'],Insp_seq[['Period_T','Exp_T']],label = ['Period_T','Exp_T'])
axes[1].plot(Insp_seq['In'],Insp_seq['Insp_T'],label = 'Insp_T')
axes[1].set(ylabel='Duration (s)',xlabel='Time (s)')
axes[1].grid()
axes[1].legend()

axes[2].plot(Insp_seq['Ex'],Insp_seq[['dPT','relD']],label = ['Seq. Diff. Period','Seq. Diff. Depth'])
axes[2].set(xlim=[250,310],ylabel='Ratios',xlabel='Time (s)')
axes[2].grid()
axes[2].legend()
plt.show()


In this time interval, the participant's breathing is relatively regular, shallow, and nearly even in duration of inspiration and expiration phases. This quality of consistently can be seen in the respiratory features plotted, little variation in the durations ins correspondingly low difference ratios from breath to breath for almost this whole sequence. We can use these features to look at longer intervals of time.  

## Plot respiration features against event information
Data session 4 and 5 were recorded from two people at the start of a little concert of Norwegian fiddle music. We can look at the respiration features against what was happening in the concert, moment by moment, described inthe Concert_Events_Structure.csv file.

### Breathing and Listening

In [ ]:
Events = pd.read_csv(demofile_loc+'timing/Concert_Events_Structure.csv')

Events['Duration']=Events['TIME'].diff().shift(-1)
events_pallet = {'Applause':'red', 'Speech':'blue', 'Prime':'green', 'Music':'yellow'}

Now plot the duration of each respiratory cycle with the concurrent events colour coded.

In [ ]:
fig,(ax) = plt.subplots(1,1,figsize=(12,3),sharey=True)
ax.plot(Insp_seq['Ex'],Insp_seq['Period_T'],label = ['Period_T'])
ax.plot(Insp_seq['Ex'],Insp_seq['Period_T'].rolling(7,center=True).median(),label = ['Period_T'])
ax.grid()
for cat in events_pallet.keys(): #'Vocal?',
    ax.axvspan(-20,-15, facecolor=events_pallet[cat],alpha = 0.2,label=cat)
ax.legend(loc='upper right') 
for i,row in Events.iloc[:20,:].iterrows():
    ax.axvspan(row['TIME'],row['TIME']+row['Duration'], facecolor=events_pallet[row['LABEL']],alpha = 0.2)
ax.set(xlim = [0,1000],ylim = [0,13],ylabel='Respiration Period (s)',xlabel = 'Time (s)')
plt.title('Respiratory period duration during the start of a concert performance') 
plt.show()

The duration of individual breaths are extremely variable in the first few seconds of this recording interval, before the group applauded and the performers started addressing the group verbally. 

Through the speach intervals (blue) and music intervals (yellow) the breathing period looks to be relatively stable for many breaths at a time, interupted by big deviations here and there. 

This change in consistency should also be visible in the sequential difference features.

In [ ]:
fig,(ax) = plt.subplots(1,1,figsize=(12,3),sharey=True)
ax.plot(Insp_seq['Ex'],Insp_seq['dPT'],label = ['Period_T'])
ax.plot(Insp_seq['Ex'],Insp_seq['dPT'].rolling(7,center=True).median(),label = ['Period_T'])
ax.grid()
for cat in events_pallet.keys(): #'Vocal?',
    ax.axvspan(-20,-15, facecolor=events_pallet[cat],alpha = 0.2,label=cat)
ax.legend(loc='upper right') 
for i,row in Events.iloc[:20,:].iterrows():
    ax.axvspan(row['TIME'],row['TIME']+row['Duration'], facecolor=events_pallet[row['LABEL']],alpha = 0.2)
ax.set(ylabel='Resp Period sequential difference',xlabel='Time (s)',xlim = [0,1000],ylim = [0,3])

plt.title('Sequential difference of respiratory period duration during the start of a concert performance') 

plt.show()

We can look at the same participant during an interval of dancing tooEvents = pd.read_csv('./demodata/timing/Concert_Events_Structure.csv')

### Breathing and Dancing

In [ ]:
Events = pd.read_csv(demofile_loc+'timing/Dancing_Events_Structure.csv')
Events = Events.loc[Events.Structure.str.len()>0,:].copy()
Events['Duration']=Events['TIME'].diff().shift(-1)
events_pallet = {'D':'blue','T':'green','M':'yellow','A':'red', }

In [ ]:
recID = 7
V = pd.read_csv(df_datafiles.loc[recID,'RESPloc'])
V['DateTime'] = pd.to_datetime(V['DateTime'])
V['Time_s'] = (V['DateTime']-V.loc[0,'DateTime']).dt.total_seconds()
Resp = V.set_index('Time_s').copy()
Breaths = rp2.breath_cycles(rp2.EQ_Inspiration_Extract(Resp['Breathing']),Resp['Breathing'])
Insp_seq = rp2.quickcarresp(Breaths)
Insp_seq

In [ ]:
fig,(axes) = plt.subplots(3,1,figsize=(12,5),sharex=True)
axes[0].scatter(Insp_seq['In'],Insp_seq['In_C'],label = 'Inspiration onset')
axes[0].scatter(Insp_seq['Ex'],Insp_seq['Ex_C'],c='r',label = 'Expiration onset')
Resp['Breathing'].plot(ax = axes[0],label='Respiration wave')
axes[0].set(ylabel = 'Resp wave')

axes[0].grid()
axes[1].plot(Insp_seq['Ex'],Insp_seq[['Period_T','Exp_T']],label = ['Period_T','Exp_T'])
axes[1].plot(Insp_seq['In'],Insp_seq['Insp_T'],label = 'Insp_T')
axes[1].set(ylabel='Duration (s)',xlabel='Time (s)')
axes[1].grid()
axes[1].legend()

axes[2].plot(Insp_seq['Ex'],Insp_seq[['dPT','relD']],label = ['Seq. Diff. Period','Seq. Diff. Depth'])
axes[2].set(xlim=[250,310],ylabel='Ratios',xlabel='Time (s)')
axes[2].grid()
axes[2].legend()
plt.show()

In [ ]:
fig,(ax) = plt.subplots(1,1,figsize=(12,3),sharey=True)
ax.plot(Insp_seq['Ex'],Insp_seq['Period_T'],label = ['Period_T'])
ax.plot(Insp_seq['Ex'],Insp_seq['Period_T'].rolling(7,center=True).median(),label = ['Period_T'])
ax.grid()
for cat in events_pallet.keys(): #'Vocal?',
    ax.axvspan(-20,-15, facecolor=events_pallet[cat],alpha = 0.2,label=cat)
ax.legend(loc='upper right') 
for i,row in Events.iloc[:35,:].iterrows():
    if row['Structure'][0] in events_pallet.keys():
        ax.axvspan(row['TIME'],row['TIME']+row['Duration'], facecolor=events_pallet[row['Structure'][0]],alpha = 0.2)
ax.set(xlim = [0,750],ylim = [0,13],ylabel='Respiration Period (s)',xlabel = 'Time (s)')
plt.title('Respiratory period duration during the dance lesson') 
plt.show()

In [ ]:
fig,(ax) = plt.subplots(1,1,figsize=(12,3),sharey=True)
ax.plot(Insp_seq['Ex'],Insp_seq['dPT'],label = ['Period_T'])
ax.plot(Insp_seq['Ex'],Insp_seq['dPT'].rolling(7,center=True).median(),label = ['Period_T'])
ax.grid()
for cat in events_pallet.keys(): #'Vocal?',
    ax.axvspan(-20,-15, facecolor=events_pallet[cat],alpha = 0.2,label=cat)
ax.legend(loc='upper right') 
for i,row in Events.iloc[:35,:].iterrows():
    if row['Structure'][0] in events_pallet.keys():
        ax.axvspan(row['TIME'],row['TIME']+row['Duration'], facecolor=events_pallet[row['Structure'][0]],alpha = 0.2)
ax.set(xlim = [0,750],ylim = [0,3],ylabel='Resp Period sequential difference',xlabel = 'Time (s)')

plt.title('Sequential difference of respiratory period duration during the start of a dance lesson') 

plt.show()

### Breathing and Singing

In [ ]:
Events = pd.read_csv(demofile_loc+'timing/Singing_Events_Structure.csv')
Events = Events.loc[Events.Type.str.len()>0,:].copy()
Events['Duration']=Events['TIME'].diff().shift(-1)
events_pallet = {'D':'blue','T':'green','M':'yellow','A':'red', }

In [ ]:
recID = 3
V = pd.read_csv(df_datafiles.loc[recID,'RESPloc'])
V['DateTime'] = pd.to_datetime(V['DateTime'])
V['Time_s'] = (V['DateTime']-V.loc[0,'DateTime']).dt.total_seconds()
Resp = V.set_index('Time_s').copy()
Breaths = rp2.breath_cycles(rp2.EQ_Inspiration_Extract(Resp['Breathing']),Resp['Breathing'])
Insp_seq = rp2.quickcarresp(Breaths)
Insp_seq

In [ ]:
fig,(axes) = plt.subplots(3,1,figsize=(12,5),sharex=True)
axes[0].scatter(Insp_seq['In'],Insp_seq['In_C'],label = 'Inspiration onset')
axes[0].scatter(Insp_seq['Ex'],Insp_seq['Ex_C'],c='r',label = 'Expiration onset')
Resp['Breathing'].plot(ax = axes[0],label='Respiration wave')
axes[0].set(ylabel = 'Resp wave')

axes[0].grid()
axes[1].plot(Insp_seq['Ex'],Insp_seq[['Period_T','Exp_T']],label = ['Period_T','Exp_T'])
axes[1].plot(Insp_seq['In'],Insp_seq['Insp_T'],label = 'Insp_T')
axes[1].set(ylabel='Duration (s)',xlabel='Time (s)')
axes[1].grid()
axes[1].legend()

axes[2].plot(Insp_seq['Ex'],Insp_seq[['dPT','relD']],label = ['Seq. Diff. Period','Seq. Diff. Depth'])
axes[2].set(xlim=[250,310],ylabel='Ratios',xlabel='Time (s)')
axes[2].grid()
axes[2].legend()
plt.show()

In [ ]:
fig,(ax) = plt.subplots(1,1,figsize=(12,3),sharey=True)
ax.plot(Insp_seq['Ex'],Insp_seq['Period_T'],label = ['Period_T'])
ax.plot(Insp_seq['Ex'],Insp_seq['Period_T'].rolling(7,center=True).median(),label = ['Period_T'])
ax.grid()
for cat in events_pallet.keys(): #'Vocal?',
    ax.axvspan(-20,-15, facecolor=events_pallet[cat],alpha = 0.2,label=cat)
ax.legend(loc='upper right') 
for i,row in Events.iloc[:35,:].iterrows():
    if row['Type'][0] in events_pallet.keys():
        ax.axvspan(row['TIME'],row['TIME']+row['Duration'], facecolor=events_pallet[row['Type'][0]],alpha = 0.2)
ax.set(xlim = [0,500],ylim = [0,13],ylabel='Respiration Period (s)',xlabel = 'Time (s)')
plt.title('Respiratory period duration during Carole Singing') 
plt.show()

In [ ]:
fig,(ax) = plt.subplots(1,1,figsize=(12,3),sharey=True)
ax.plot(Insp_seq['Ex'],Insp_seq['dPT'],label = ['Period_T'])
ax.plot(Insp_seq['Ex'],Insp_seq['dPT'].rolling(7,center=True).median(),label = ['Period_T'])
ax.grid
for cat in events_pallet.keys(): #'Vocal?',
    ax.axvspan(-20,-15, facecolor=events_pallet[cat],alpha = 0.2,label=cat)
ax.legend(loc='upper right') 
for i,row in Events.iloc[:35,:].iterrows():
    if row['Type'][0] in events_pallet.keys():
        ax.axvspan(row['TIME'],row['TIME']+row['Duration'], facecolor=events_pallet[row['Type'][0]],alpha = 0.2)
ax.set(xlim = [0,500],ylim = [0,3],ylabel='Resp Period sequential difference',xlabel = 'Time (s)')
plt.title('Sequential difference of respiratory period during Carole Singing')
plt.show()